In [1]:
import awswrangler as wr
from datetime import datetime
import logging
import json

# Configura o logger para a Lambda
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Constantes de configuração (melhor se vierem de variáveis de ambiente em produção)
S3_FILE_PATH = "s3://recepcao-taxa-de-juros/interest_rate.csv"
S3_PARQUET_PATH = "s3://walter-araujo-bucket-exemplo/interest_rate/"
DATABASE_NAME = "walter_araujo_database_sor"
TABLE_NAME = "interest_rate"

def lambda_handler(event, context):
    logger.info("Iniciando processamento da Lambda.")
    
    try:
        logger.info(f"Lendo CSV de {S3_FILE_PATH}")
        df = wr.s3.read_csv(path=S3_FILE_PATH, dtype=str)

        df["data_ingestao"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        logger.info("Coluna de data de ingestão adicionada.")

        logger.info(f"Escrevendo dados em formato Parquet em {S3_PARQUET_PATH}")
        wr.s3.to_parquet(
            df=df,
            path=S3_PARQUET_PATH,
            dataset=True,
            mode="overwrite",
            database=DATABASE_NAME,
            table=TABLE_NAME,
            dtype={col: "string" for col in df.columns}
        )
        
        logger.info("Processamento concluído com sucesso.")
        return {
            'statusCode': 200,
            'body': json.dumps('Processamento concluído com sucesso.')
        }
    
    except Exception as e:
        logger.error(f"Erro durante a execução: {str(e)}", exc_info=True)
        return {
            'statusCode': 500,
            'body': json.dumps(f'Erro: {str(e)}')
        }

# Execução local para testes
if __name__ == "__main__":
    from traceback import format_exc
    try:
        lambda_handler(None, None)
    except Exception:
        print(format_exc())
        exit(1)


Erro durante a execução: An error occurred (EntityNotFoundException) when calling the CreateTable operation: Database walter_araujo_database_sor not found.
Traceback (most recent call last):
  File "C:\Users\waltujo\AppData\Local\Temp\ipykernel_2340\766362827.py", line 27, in lambda_handler
    wr.s3.to_parquet(
  File "c:\Users\waltujo\AppData\Local\Programs\Python\Python311\Lib\site-packages\awswrangler\_config.py", line 712, in wrapper
    return function(**args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\waltujo\AppData\Local\Programs\Python\Python311\Lib\site-packages\awswrangler\_utils.py", line 179, in inner
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\waltujo\AppData\Local\Programs\Python\Python311\Lib\site-packages\awswrangler\s3\_write_parquet.py", line 723, in to_parquet
    return strategy.write(
           ^^^^^^^^^^^^^^^
  File "c:\Users\waltujo\AppData\Local\Programs\Python\Python311\Lib\site-packages\awswrangler\s3\_write.py", lin